<a href="https://colab.research.google.com/github/Hashara/CS4622-Machine-Learning-Project/blob/main/CS4622_Machine_Learning_ProjectV6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libararies

In [ ]:
 !pip install eli5 pdpbox category_encoders

     |████████████████████████████████| 106 kB 6.6 MB/s 
     |████████████████████████████████| 34.0 MB 40 kB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 13.1 MB 7.0 kB/s 
  Created wheel for pdpbox: filename=PDPbox-0.2.1-py3-none-any.whl size=35758225 sha256=60e8d7305438c4676f2f72833d051a8621db96d4b79c9e18d71f5b208e13d776
  Stored in directory: /root/.cache/pip/wheels/f4/d0/1a/b80035625c53131f52906a6fc4dd690d8efd2bf8af6a4015eb
Successfully built pdpbox
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
! pip install pandas==0.25

     |████████████████████████████████| 10.4 MB 5.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires pandas>=1.0, but you have pandas 0.25.0 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 0.25.0 which is incompatible.
fbprophet 0.7.1 requires pandas>=1.0.4, but you have pandas 0.25.0 which is incompatible.


In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
import statsmodels.api as sm
from sklearn import metrics

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
import pandas_profiling
from sklearn.metrics import classification_report

In [75]:
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

# Read CSVs

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
df_train_val = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/training_set_values.csv')
df_test_val = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/test_set_values.csv')
df_train_label = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/training_set_labels.csv')
df_submission = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/SubmissionFormat.csv')

In [78]:
df_train_val.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [79]:
df_train_label['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [80]:
df_train_val.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

In [81]:
pandas_profiling.ProfileReport(df_train_val)

TypeError: ignored

# Feature engineering

## Missing Values

In [89]:
cols_with_missing = [col for col in df_train_val.columns
                     if df_train_val[col].isnull().any()]

In [90]:
cols_with_missing

['funder',
 'installer',
 'subvillage',
 'public_meeting',
 'scheme_management',
 'scheme_name',
 'permit']

## Fill missing values

### scheme_management

In [91]:
# df_train_val.scheme_management.describe()

In [92]:
# df_train_val.scheme_management.value_counts()

In [93]:
# df_train_val.scheme_management = df_train_val.scheme_management.fillna('VWC')
# df_test_val.scheme_management = df_test_val.scheme_management.fillna('VWC')

In [94]:
# df_train_val.loc[df_train_val.scheme_management == 'Trust', 'scheme_management'] = 'Other'
# df_test_val.loc[df_test_val.scheme_management == 'Trust', 'scheme_management'] = 'Other'

# df_train_val.loc[df_train_val.scheme_management == 'SWC', 'scheme_management'] = 'Other'
# df_test_val.loc[df_test_val.scheme_management == 'SWC', 'scheme_management'] = 'Other'

### permit

In [95]:
# df_train_val.permit.describe()

In [96]:
# df_train_val.permit.value_counts()

In [97]:
# df_train_val.permit = df_train_val.permit.fillna(False)
# df_test_val.permit = df_test_val.permit.fillna(False)

### public_meeting

In [98]:
# df_train_val.public_meeting.describe()

In [99]:
# df_train_val.public_meeting.value_counts()

In [100]:
# df_train_val.public_meeting = df_train_val.permit.fillna(False)
# df_test_val.public_meeting = df_test_val.permit.fillna(False)

### installer

In [101]:
df_train_val.installer.describe()

count     55745
unique     2145
top         DWE
freq      17402
Name: installer, dtype: object

In [102]:
df_train_val.installer.value_counts()

DWE                           17402
Government                     1825
RWE                            1206
Commu                          1060
DANIDA                         1050
                              ...  
Water authority                   1
Saidi Halfani                     1
Care  international               1
Africaone Ltd                     1
MKONGO BUILDING CONTRACTOR        1
Name: installer, Length: 2145, dtype: int64

In [103]:
df_test_val.loc[df_train_val.groupby('installer').installer.transform('count').lt(50), 'installer'] = "Other" 
df_test_val.loc[df_test_val.groupby('installer').installer.transform('count').lt(50), 'installer'] = "Other"
df_train_val.loc[df_train_val.groupby('installer').installer.transform('count').lt(50), 'installer'] = "Other"    

In [104]:
df_train_val.installer = df_train_val.installer.fillna('DWE')
df_test_val.installer = df_test_val.installer.fillna('DWE')

## Drop missing values

In [105]:
cols_with_missing = [col for col in df_test_val.columns
                     if df_test_val[col].isnull().any()]
cols_with_missing

['funder',
 'subvillage',
 'public_meeting',
 'scheme_management',
 'scheme_name',
 'permit']

In [106]:
# drop missing values
df_train_val = df_train_val.drop(columns = cols_with_missing)
df_test_val = df_test_val.drop(columns = cols_with_missing)

## Exploring data

In [107]:
df_train_val.columns

Index(['id', 'amount_tsh', 'date_recorded', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'region',
       'region_code', 'district_code', 'lga', 'ward', 'population',
       'recorded_by', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [108]:
df_train_val.head()

,id,amount_tsh,date_recorded,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,1399,Other,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [109]:
df_train_val.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [110]:
df_train_val.corr()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,1.000000,-0.005321,-0.004692,-0.001348,0.001718,-0.002629,-0.003028,-0.003044,-0.002813,-0.002082
amount_tsh,-0.005321,1.000000,0.076650,0.022134,-0.052670,0.002944,-0.026813,-0.023599,0.016288,0.067915
gps_height,-0.004692,0.076650,1.000000,0.149155,-0.035751,0.007237,-0.183521,-0.171233,0.135003,0.658727
longitude,-0.001348,0.022134,0.149155,1.000000,-0.425802,0.023873,0.034197,0.151398,0.086590,0.396732
latitude,0.001718,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.221018,-0.201020,-0.022152,-0.245278
num_private,-0.002629,0.002944,0.007237,0.023873,0.006837,1.000000,-0.020377,-0.004478,0.003818,0.026056
region_code,-0.003028,-0.026813,-0.183521,0.034197,-0.221018,-0.020377,1.000000,0.678602,0.094088,0.031724
district_code,-0.003044,-0.023599,-0.171233,0.151398,-0.201020,-0.004478,0.678602,1.000000,0.061831,0.048315
population,-0.002813,0.016288,0.135003,0.086590,-0.022152,0.003818,0.094088,0.061831,1.000000,0.260910
construction_year,-0.002082,0.067915,0.658727,0.396732,-0.245278,0.026056,0.031724,0.048315,0.260910,1.000000


## Adding new features

In [111]:
df_train_val.head()

,id,amount_tsh,date_recorded,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,1399,Other,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


### Number of years

In [112]:
df_train_val["year_recorded"] = df_train_val["date_recorded"].astype(str).str[0:4].astype(int)
df_test_val["year_recorded"] = df_test_val["date_recorded"].astype(str).str[0:4].astype(int)

In [113]:
df_train_val["number_of_years"] = df_train_val["year_recorded"] - df_train_val["construction_year"]
df_test_val["number_of_years"] = df_test_val["year_recorded"] - df_test_val["construction_year"]

In [114]:
df_train_val["number_of_years"].value_counts()

 2011    13343
 2012     5142
 3        2740
 1        2303
 2013     2210
         ...  
-1           1
-3           1
-2           1
 2002        1
-7           1
Name: number_of_years, Length: 65, dtype: int64

In [115]:
x = pd.DataFrame()
x["number_of_years"] = df_train_val["number_of_years"]
x = x.drop(x[x.number_of_years > 2000].index)
x["number_of_years"].describe()

count    38691.000000
mean        15.355742
std         12.492673
min         -7.000000
25%          5.000000
50%         13.000000
75%         25.000000
max         53.000000
Name: number_of_years, dtype: float64

In [116]:
df_train_val.loc[df_train_val.number_of_years > 2000, 'number_of_years'] = 15
df_test_val.loc[df_train_val.number_of_years > 2000, 'number_of_years'] = 15

In [117]:
df_train_val.head()

,id,amount_tsh,date_recorded,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year_recorded,number_of_years
0,69572,6000.0,2011-03-14,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2011,12
1,8776,0.0,2013-03-06,1399,Other,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3
2,34310,25.0,2013-02-25,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,2013,4
3,67743,0.0,2013-01-28,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,27
4,19728,0.0,2011-07-13,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,15


In [118]:
df_train_val["status_group"] = df_train_label["status_group"]

### GPS height

In [119]:
# df_train_val["gps_height"].describe()

In [120]:
# df_train_val["gps_height"].value_counts()

In [121]:
# df_train_val.hist(column=['gps_height'],bins=5, figsize=(8,6))

In [122]:
# #group 
# category = pd.cut(df_train_val.gps_height,bins=[-100,500,1000,1500,2000,2500,3000],labels=[0,1,2,3,4,5])
# df_train_val.insert(5,'gps_height_group',category)

# category = pd.cut(df_test_val.gps_height,bins=[-100,500,1000,1500,2000,2500,3000],labels=[0,1,2,3,4,5])
# df_test_val.insert(5,'gps_height_group',category)

### amount_tsh

In [123]:
# df_train_val["amount_tsh"].describe()

In [124]:
# df_train_val["amount_tsh"].value_counts()

In [125]:
# x = pd.DataFrame()
# x["amount_tsh"] = df_train_val["amount_tsh"]
# x = x.drop(x[x.amount_tsh < 10].index)
# x["amount_tsh"].describe()

In [126]:
# x["amount_tsh"].value_counts().sort_index()

In [127]:
# df_train_val.hist(column=['amount_tsh'],bins=5, figsize=(8,6))

In [128]:
# #group 
# category = pd.cut(df_train_val.amount_tsh,bins=[-100,20000,30000,50000,100000, 200000,400000],labels=[0,1,2,3,4,5])
# df_train_val.insert(5,'amount_tsh_group',category)

# category = pd.cut(df_test_val.amount_tsh,bins=[-100,20000,30000,50000,100000, 200000,400000],labels=[0,1,2,3,4,5])
# df_test_val.insert(5,'amount_tsh_group',category)

### Population

In [129]:
# df_train_val["population"].describe()

In [130]:
# df_train_val["population"].value_counts()

In [131]:
# x = pd.DataFrame()
# x["population"] = df_train_val["population"]
# x = x.drop(x[x.population < 10].index)
# x["population"].describe()

In [132]:
# x["population"].value_counts().sort_index()

In [133]:
# x.hist(column=['population'],bins=20, figsize=(20,6))

In [134]:
# df_train_val.loc[df_train_val.number_of_years < 10, 'population'] = 346
# df_test_val.loc[df_test_val.number_of_years < 10, 'population'] = 346

In [135]:
# [col for col in df_train_val.columns
#                      if df_train_val[col].isnull().any()]

In [136]:
# # group 
# category = pd.cut(df_train_val.population,bins=[-100,250,500,1000,2500,5000,100000],labels=[0,1,2,3,4,5])
# df_train_val.insert(6,'population_group',category)

# category = pd.cut(df_test_val.population,bins=[-100,250,500,1000,2500,5000,100000],labels=[0,1,2,3,4,5])
# df_test_val.insert(6,'population_group',category)

In [137]:
# [col for col in df_train_val.columns
#                      if df_train_val[col].isnull().any()]

In [138]:
# df_train_val.population.describe()

In [139]:
# df_train_val.population_group.value_counts()

## Encoding

In [140]:
# object_cols = [ 'basin', 'lga', 'ward','extraction_type','extraction_type_group',
#                'extraction_type_class','management', 'management_group', 'payment',	'payment_type',
#                'water_quality',	'quality_group',	'quantity',	'quantity_group',	'source',	'source_type',
#                'source_class',	'waterpoint_type',	'waterpoint_type_group']
# object_cols = ['payment_type', 'source_type', 'quantity_group','quality_group', 'management', 
#                'extraction_type_group', 'installer']
# object_cols = ['payment_type', 'source_type', 'quantity_group','quality_group', 'management', 
#                'extraction_type_group', 'installer', 'basin']
object_cols = ['basin', 'region', 'lga',  'extraction_type_group', 'management_group',
       'payment', 'water_quality', 'quality_group', 'quantity_group', 'source',
       'source_class', 'waterpoint_type']
      #  , 'installer']

In [ ]:
# df_train_val = label_X_train.copy()
# df_test_val = label_X_test.copy()

In [61]:
df_test_val[object_cols]

,basin,region,lga,extraction_type_group,management_group,payment,water_quality,quality_group,quantity_group,source,source_class,waterpoint_type_group
0,Internal,Manyara,Mbulu,other,parastatal,never pay,soft,good,seasonal,rainwater harvesting,surface,other
1,Pangani,Arusha,Arusha Rural,gravity,user-group,never pay,soft,good,insufficient,spring,groundwater,communal standpipe
2,Internal,Singida,Singida Rural,other,user-group,never pay,soft,good,insufficient,rainwater harvesting,surface,other
3,Ruvuma / Southern Coast,Lindi,Liwale,other,user-group,unknown,soft,good,dry,shallow well,groundwater,other
4,Ruvuma / Southern Coast,Ruvuma,Mbinga,gravity,user-group,pay monthly,soft,good,enough,spring,groundwater,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...
14845,Wami / Ruvu,Pwani,Bagamoyo,mono,user-group,never pay,soft,good,enough,river,surface,communal standpipe
14846,Pangani,Tanga,Kilindi,nira/tanira,user-group,pay annually,salty,salty,insufficient,shallow well,groundwater,hand pump
14847,Internal,Singida,Singida Rural,gravity,user-group,never pay,soft,good,insufficient,dam,surface,communal standpipe
14848,Lake Nyasa,Ruvuma,Songea Rural,gravity,user-group,never pay,soft,good,insufficient,river,surface,communal standpipe


In [141]:
# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
df_train_val[object_cols] = ordinal_encoder.fit_transform(df_train_val[object_cols])
df_test_val[object_cols] = ordinal_encoder.transform(df_test_val[object_cols])

### One hot encoding

In [63]:
one_hot_cols = df_train_val.select_dtypes(["object", "category"]).columns.tolist()

In [64]:
one_hot_cols

['date_recorded',
 'installer',
 'wpt_name',
 'ward',
 'recorded_by',
 'extraction_type',
 'extraction_type_class',
 'management',
 'payment_type',
 'quantity',
 'source_type',
 'waterpoint_type',
 'status_group']

In [65]:
one_hot_cols.remove('date_recorded')
one_hot_cols.remove('recorded_by')
one_hot_cols.remove('status_group')

In [66]:
one_hot_cols

['installer',
 'wpt_name',
 'ward',
 'extraction_type',
 'extraction_type_class',
 'management',
 'payment_type',
 'quantity',
 'source_type',
 'waterpoint_type']

In [67]:
object_cols = ['basin', 'waterpoint_type_group']

In [68]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(one_hot_encoder.fit_transform(df_train_val[object_cols]))
OH_cols.index = df_train_val.index

OH_cols.columns = one_hot_encoder.get_feature_names(object_cols)

num_df = df_train_val.drop(object_cols, axis=1)
df_train_val = pd.concat([num_df, OH_cols], axis=1)

OH_cols = pd.DataFrame(one_hot_encoder.transform(df_test_val[object_cols]))
OH_cols.index = df_test_val.index

OH_cols.columns = one_hot_encoder.get_feature_names(object_cols)

num_df = df_test_val.drop(object_cols, axis=1)
df_test_val = pd.concat([num_df, OH_cols], axis=1)

In [69]:
df_test_val

,id,amount_tsh,date_recorded,gps_height,installer,longitude,latitude,wpt_name,num_private,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,year_recorded,number_of_years,basin_0.0,basin_1.0,basin_2.0,basin_3.0,basin_4.0,basin_5.0,basin_6.0,basin_7.0,basin_8.0,waterpoint_type_group_0.0,waterpoint_type_group_1.0,waterpoint_type_group_2.0,waterpoint_type_group_3.0,waterpoint_type_group_4.0,waterpoint_type_group_5.0
0,50785,0.0,2013-02-04,1996,Other,35.290799,-4.059696,Dinamu Secondary School,0,8.0,21,3,63.0,Bashay,321,GeoData Consultants Ltd,2012,other,6.0,other,parastatal,2.0,0.0,never pay,6.0,2.0,seasonal,3.0,5.0,rainwater harvesting,1.0,other,2013,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,51630,0.0,2013-02-04,1569,Other,36.656709,-3.309214,Kimnyak,0,0.0,2,2,0.0,Kimnyaki,300,GeoData Consultants Ltd,2000,gravity,1.0,gravity,vwc,4.0,0.0,never pay,6.0,2.0,insufficient,2.0,8.0,spring,0.0,communal standpipe,2013,13,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,17168,0.0,2013-02-01,1567,DWE,34.767863,-5.004344,Puma Secondary,0,18.0,13,2,109.0,Puma,500,GeoData Consultants Ltd,2010,other,6.0,other,vwc,4.0,0.0,never pay,6.0,2.0,insufficient,2.0,5.0,rainwater harvesting,1.0,other,2013,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,45559,0.0,2013-01-22,267,Other,38.058046,-9.418672,Kwa Mzee Pange,0,7.0,80,43,49.0,Mkutano,250,GeoData Consultants Ltd,1987,other,6.0,other,vwc,4.0,6.0,unknown,6.0,2.0,dry,0.0,7.0,shallow well,0.0,other,2013,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,49871,500.0,2013-03-27,1260,Other,35.006123,-10.950412,Kwa Mzee Turuka,0,16.0,10,3,61.0,Mbinga Urban,60,GeoData Consultants Ltd,2000,gravity,1.0,gravity,water board,4.0,3.0,monthly,6.0,2.0,enough,1.0,8.0,spring,0.0,communal standpipe,2013,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,34,Da,38.852669,-6.582841,Kwambwezi,0,14.0,6,1,3.0,Yombo,20,GeoData Consultants Ltd,1988,mono,4.0,motorpump,vwc,4.0,0.0,never pay,6.0,2.0,enough,1.0,6.0,river/lake,1.0,communal standpipe,2011,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
14846,18990,1000.0,2011-03-21,0,Other,37.451633,-5.350428,Bonde La Mkondoa,0,20.0,4,7,33.0,Mvungwe,2960,GeoData Consultants Ltd,1994,nira/tanira,5.0,handpump,vwc,4.0,2.0,annually,4.0,4.0,insufficient,2.0,7.0,shallow well,0.0,hand pump,2011,17,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14847,28749,0.0,2013-03-04,1476,Other,34.739804,-4.585587,Bwawani,0,18.0,13,2,109.0,Ughandi,200,GeoData Consultants Ltd,2010,gravity,1.0,gravity,vwc,4.0,0.0,never pay,6.0,2.0,insufficient,2.0,0.0,dam,1.0,communal standpipe,2013,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
14848,33492,0.0,2013-02-18,998,DWE,35.432732,-10.584159,Kwa John,0,16.0,10,2,111.0,Maposeni,150,GeoData Consultants Ltd,2009,gravity,1.0,gravity,vwc,4.0,0.0,never pay,6.0,2.0,insufficient,2.0,6.0,river/lake,1.0,communal standpipe,2013,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Ploting data

In [70]:
plt.figure(figsize=(20,20))

<Figure size 1440x1440 with 0 Axes>

In [71]:
sns.relplot(x="id",y="number_of_years",hue="status_group",data=df_train_val);
plt.show()

In [72]:
sns.relplot(x="id",y="amount_tsh",hue="status_group",data=df_train_val);

In [112]:
sns.relplot(x="id",y="gps_height",hue="status_group",data=df_train_val);

In [113]:
sns.relplot(x="id",y="num_private",hue="status_group",data=df_train_val);

In [114]:
sns.relplot(x="id",y="basin",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="region",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="region_code",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="district_code",hue="status_group",data=df_train_val);

In [ ]:
df_train_val.status_group.value_counts().plot.barh()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='status_group', hue='number_of_years', data= df_train_val)

# Clean dataset

In [ ]:
df_train_val.corr()

In [61]:
df_train_val.columns

Index(['id', 'amount_tsh', 'date_recorded', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'region',
       'region_code', 'district_code', 'lga', 'ward', 'population',
       'recorded_by', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'year_recorded', 'number_of_years', 'status_group'],
      dtype='object')

In [142]:
object_cols
# 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
#        'basin', 'region', 'district_code', 'lga', 'population',
#        'construction_year', 'extraction_type_group', 'management_group',
#        'payment', 'water_quality', 'quality_group', 'quantity_group', 'source',
#        'source_class', 'waterpoint_type_group

['basin',
 'region',
 'lga',
 'extraction_type_group',
 'management_group',
 'payment',
 'water_quality',
 'quality_group',
 'quantity_group',
 'source',
 'source_class',
 'waterpoint_type']

In [143]:
df_train_clean = df_train_val[[ 'gps_height', 'longitude', 'latitude', 'num_private',
       'basin', 'region', 'district_code', 'lga', 'population',
       'construction_year', 'extraction_type_group', 'management_group',
       'payment', 'water_quality',  'quantity_group', 'source',
       'waterpoint_type']]
df_test_clean = df_test_val[[ 'gps_height', 'longitude', 'latitude', 'num_private',
       'basin', 'region', 'district_code', 'lga', 'population',
       'construction_year', 'extraction_type_group', 'management_group',
       'payment', 'water_quality',  'quantity_group', 'source',
       'waterpoint_type']]
df_train_label_clean = df_train_label['status_group']

In [ ]:
# df_train_clean = df_train_val.drop(columns=['id', 'amount_tsh', 'date_recorded', 'gps_height', 'wpt_name', 
#                                             # 'basin',
#                                             'region',
#                                             'recorded_by',
#        'lga', 
#        'ward',  'population',
#        'construction_year', 'extraction_type', 
#        'extraction_type_class',  'management_group', 'payment',
#        'water_quality',  'quantity', 
#        'source', 'source_class',
#        'waterpoint_type',
#         'waterpoint_type_group', 
#         'year_recorded',
#        'status_group'])
# df_test_clean = df_test_val.drop(columns=['id', 'amount_tsh', 'date_recorded', 'gps_height', 'wpt_name', 
#                                           # 'basin',
#                                           'region', 
#                                           'recorded_by',
#        'lga', 
#        'ward',   'population',
#        'construction_year', 'extraction_type', 
#        'extraction_type_class',  'management_group', 'payment',
#        'water_quality',  'quantity', 
#        'source', 'source_class',
#        'waterpoint_type',
#         'waterpoint_type_group', 
#         'year_recorded',
#        ])
# df_train_label_clean = df_train_val["status_group"]


In [144]:
df_test_clean.head()

,gps_height,longitude,latitude,num_private,basin,region,district_code,lga,population,construction_year,extraction_type_group,management_group,payment,water_quality,quantity_group,source,waterpoint_type
0,1996,35.290799,-4.059696,0,0.0,8.0,3,63.0,321,2012,6.0,2.0,0.0,6.0,3.0,5.0,6.0
1,1569,36.656709,-3.309214,0,5.0,0.0,2,0.0,300,2000,1.0,4.0,0.0,6.0,2.0,8.0,1.0
2,1567,34.767863,-5.004344,0,0.0,18.0,2,109.0,500,2010,6.0,4.0,0.0,6.0,2.0,5.0,6.0
3,267,38.058046,-9.418672,0,7.0,7.0,43,49.0,250,1987,6.0,4.0,6.0,6.0,0.0,7.0,6.0
4,1260,35.006123,-10.950412,0,7.0,16.0,3,61.0,60,2000,1.0,4.0,3.0,6.0,1.0,8.0,1.0


# Under sampling and oversampling

In [147]:
X_train, X_test, y_train, y_test = train_test_split(df_train_clean, df_train_label_clean, test_size=0.1, random_state=10, shuffle=True)

In [108]:
# sns.set(font_scale=1.4)
# y_train.value_counts().plot(kind='bar', figsize=(7, 6), rot=0)

plt.figure()
sns.barplot(y_train.index, y_train.values, alpha=0.8)
# plt.title('Starbucks in top 10 cities in the World')
# plt.ylabel('Number of Occurrences', fontsize=12)
# plt.xlabel('city', fontsize=12)
plt.show()

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [ ]:
# undersample = NearMiss(version=2, n_neighbors=3)
# X_train, y_train = undersample.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [67]:
oversample = SMOTE()
X_train_oversampled, y_train_oversampled = oversample.fit_resample(X_train, y_train)
X_train = pd.DataFrame(X_train_oversampled, columns=X_train.columns)
y_train =  pd.DataFrame(y_train_oversampled, columns=['status_type'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# oversample = RandomOverSampler(sampling_strategy='minority')
# X_train_oversampled, y_train_oversampled = oversample.fit_resample(X_train, y_train)
# X_train = pd.DataFrame(X_train_oversampled, columns=X_train.columns)
# y_train =  pd.DataFrame(y_train_oversampled, columns=['status_type'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state = 42)
# X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
# X_train = pd.DataFrame(X_train_oversampled, columns=X_train.columns)

In [68]:
X_train

,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,district_code,lga,population,number_of_years,extraction_type_group,management_group,payment,water_quality,quality_group,quantity_group,source,source_class,waterpoint_type_group
0,500.0,147.000000,37.798787,-7.378011e+00,0.0,8.0,11.0,2.0,71.0,2000.0,25.000000,5.000000,4.000000,5.000000,6.0,2.0,2.0,7.000000,0.000000,3.000000
1,0.0,0.000000,36.361102,-6.125086e+00,0.0,8.0,2.0,3.0,43.0,0.0,15.000000,10.000000,4.000000,4.000000,4.0,4.0,1.0,3.000000,0.000000,1.000000
2,10.0,1806.000000,35.581231,-3.757245e+00,0.0,0.0,8.0,3.0,63.0,255.0,2.000000,5.000000,4.000000,4.000000,6.0,2.0,1.0,7.000000,0.000000,3.000000
3,200.0,1269.000000,37.886287,-4.390882e+00,0.0,5.0,6.0,3.0,101.0,80.0,2.000000,1.000000,4.000000,3.000000,6.0,2.0,2.0,8.000000,0.000000,1.000000
4,0.0,0.000000,33.412691,-2.986055e+00,0.0,4.0,13.0,4.0,45.0,0.0,15.000000,11.000000,4.000000,0.000000,6.0,2.0,1.0,3.000000,0.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86968,0.0,1191.263918,32.950180,-2.013060e+00,0.0,4.0,13.0,1.0,121.0,300.0,18.169485,5.226392,4.000000,0.000000,6.0,2.0,2.0,5.868041,0.226392,2.547216
86969,0.0,0.000000,31.786549,-1.152462e+00,0.0,4.0,4.0,2.0,7.0,0.0,15.000000,1.000000,4.000000,0.000000,7.0,5.0,0.0,5.000000,1.000000,1.000000
86970,0.0,0.000000,33.014706,-8.440548e+00,0.0,2.0,10.0,1.0,13.0,0.0,15.000000,10.000000,4.000000,4.000000,6.0,2.0,1.0,8.000000,0.000000,1.000000
86971,0.0,0.000000,0.000000,-2.000000e-08,0.0,4.0,13.0,2.0,54.0,0.0,15.000000,1.000000,2.803177,0.000000,6.0,2.0,2.0,3.196823,1.000000,1.000000


# Model

## Random forest

In [148]:
rfc_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
rfc_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [149]:
predictions = rfc_model.predict(X_test)

In [150]:
logreg_score_train = rfc_model.score(X_train,y_train)
print("Train Prediction Score",logreg_score_train*100)
logreg_score_test = accuracy_score(y_test,predictions)
print("Test Prediction ",logreg_score_test*100)

Train Prediction Score 71.07744107744108
Test Prediction  71.70033670033669


## XgBoost

In [ ]:
# from sklearn.model_selection import GridSearchCV
# def xgmodel(X_train, X_val, y_train, y_val):
#     if __name__ == '__main__':
    
#         param_grid = {'learning_rate': [0.075, 0.25],
#                       'max_depth': [13, 14],
#                       'n_estimators': [100, 200]}                      

#         estimator = GridSearchCV(estimator=XGBClassifier(),
#                                  param_grid=param_grid,
#                                  n_jobs=-1)

#         estimator.fit(X_train, y_train)

#         best_params = estimator.best_params_

#         print (best_params)
                                 
#         validation_accuracy = estimator.score(X_val, y_val)
#         print('Validation accuracy: ', validation_accuracy)

In [ ]:
# xgmodel(X_train, X_test, y_train, y_test)

In [151]:
xgb_model = XGBClassifier(max_depth=13,n_estimators=200,learning_rate=0.075)
xgb_model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.075, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [152]:
predictions = xgb_model.predict(X_test)

In [153]:
logreg_score_train = xgb_model.score(X_train,y_train)
print("Train Prediction Score",logreg_score_train*100)
logreg_score_test = accuracy_score(y_test,predictions)
print("Test Prediction ",logreg_score_test*100)

Train Prediction Score 93.82903105125328
Test Prediction  81.5993265993266


In [75]:

print(classification_report(y_test,predictions))

                         precision    recall  f1-score   support

             functional       0.83      0.84      0.83      3268
functional needs repair       0.39      0.52      0.45       412
         non functional       0.84      0.78      0.81      2260

               accuracy                           0.79      5940
              macro avg       0.69      0.71      0.70      5940
           weighted avg       0.80      0.79      0.80      5940



## DT

In [76]:
dt = DecisionTreeClassifier( max_leaf_nodes=600)
dt.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=600,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [77]:
predictions = dt.predict(X_test)

In [78]:
logreg_score_train = dt.score(X_train,y_train)
print("Train Prediction Score",logreg_score_train*100)
logreg_score_test = accuracy_score(y_test,predictions)
print("Test Prediction ",logreg_score_test*100)

Train Prediction Score 79.41085164361354
Test Prediction  73.18181818181819


In [79]:
print(classification_report(y_test,predictions))

                         precision    recall  f1-score   support

             functional       0.79      0.77      0.78      3268
functional needs repair       0.30      0.55      0.38       412
         non functional       0.80      0.71      0.75      2260

               accuracy                           0.73      5940
              macro avg       0.63      0.68      0.64      5940
           weighted avg       0.76      0.73      0.74      5940



## Gradient Boosting

In [ ]:
# from sklearn.model_selection import GridSearchCV
# def model(X_train, X_val, y_train, y_val):
#     if __name__ == '__main__':
    
#         param_grid = {'learning_rate': [0.075, 0.7],
#                       'max_depth': [13, 14],
#                       'min_samples_leaf': [15, 16],
#                       'max_features': [1.0],
#                       'n_estimators': [100, 200]}                      

#         estimator = GridSearchCV(estimator=GradientBoostingClassifier(),
#                                  param_grid=param_grid,
#                                  n_jobs=-1)

#         estimator.fit(X_train, y_train)

#         best_params = estimator.best_params_

#         print (best_params)
                                 
#         validation_accuracy = estimator.score(X_val, y_val)
#         print('Validation accuracy: ', validation_accuracy)

In [ ]:
# model(X_train, X_test, y_train, y_test)
# {'learning_rate': 0.075, 'max_depth': 14, 'max_features': 1.0, 'min_samples_leaf': 15, 'n_estimators': 200}
# Validation accuracy:  0.8085858585858586

In [ ]:
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

In [ ]:
# gd = GradientBoostingClassifier(n_estimators=200, learning_rate=0.075, max_depth=14,  min_samples_leaf=15, max_features=0.1)
# kfold = KFold(n_splits=20, random_state=7, shuffle=True)
# results = cross_val_score(gd, df_train_clean, df_train_label_clean, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [158]:
gd = GradientBoostingClassifier(n_estimators=200, learning_rate=0.075, max_depth=14,  min_samples_leaf=15, max_features=0.1)
# gd.fit(X_train,y_train)
gd.fit(df_train_clean, df_train_label_clean)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.075, loss='deviance', max_depth=14,
                           max_features=0.1, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=15, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [155]:
predictions = gd.predict(X_test)

In [156]:
logreg_score_train = gd.score(X_train,y_train)
print("Train Prediction Score",logreg_score_train*100)
logreg_score_test = accuracy_score(y_test,predictions)
print("Test Prediction ",logreg_score_test*100)

Train Prediction Score 89.08529741863074
Test Prediction  81.38047138047138


In [157]:
print(classification_report(y_test,predictions))

                         precision    recall  f1-score   support

             functional       0.81      0.90      0.85      3268
functional needs repair       0.60      0.30      0.40       412
         non functional       0.84      0.78      0.81      2260

               accuracy                           0.81      5940
              macro avg       0.75      0.66      0.69      5940
           weighted avg       0.81      0.81      0.81      5940



## MLPClassifier

In [ ]:
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
predictions = mlp.predict(X_test)

In [ ]:
logreg_score_train = mlp.score(X_train,y_train)
print("Train Prediction Score",logreg_score_train*100)
logreg_score_test = accuracy_score(y_test,predictions)
print("Test Prediction ",logreg_score_test*100)

Train Prediction Score 62.84137673026562
Test Prediction  63.31649831649832


# Post processing

In [ ]:
df_test_clean.head()

,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,district_code,lga,population,construction_year,extraction_type_group,management_group,payment,water_quality,quality_group,quantity_group,source,source_class,waterpoint_type_group
0,0.0,1996,35.290799,-4.059696,0,0.0,8.0,3,63.0,321,2012,6.0,2.0,0.0,6.0,2.0,3.0,5.0,1.0,5.0
1,0.0,1569,36.656709,-3.309214,0,5.0,0.0,2,0.0,300,2000,1.0,4.0,0.0,6.0,2.0,2.0,8.0,0.0,1.0
2,0.0,1567,34.767863,-5.004344,0,0.0,18.0,2,109.0,500,2010,6.0,4.0,0.0,6.0,2.0,2.0,5.0,1.0,5.0
3,0.0,267,38.058046,-9.418672,0,7.0,7.0,43,49.0,250,1987,6.0,4.0,6.0,6.0,2.0,0.0,7.0,0.0,5.0
4,500.0,1260,35.006123,-10.950412,0,7.0,16.0,3,61.0,60,2000,1.0,4.0,3.0,6.0,2.0,1.0,8.0,0.0,1.0


In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(gd, random_state=1).fit(X_test, y_test)


In [ ]:
eli5.show_weights(perm, feature_names = df_train_clean.columns.tolist())

Weight,Feature
0.0959 ± 0.0087,quantity_group
0.0423 ± 0.0043,waterpoint_type_group
0.0247 ± 0.0061,extraction_type_group
0.0228 ± 0.0048,number_of_years
0.0210 ± 0.0032,payment
0.0193 ± 0.0035,source
0.0174 ± 0.0031,latitude
0.0155 ± 0.0019,longitude
0.0137 ± 0.0045,installer
0.0130 ± 0.0059,gps_height


# Write to CSV

In [159]:
df_submission["status_group"] = gd.predict(df_test_clean)

In [ ]:
# df_submission["status_group"].replace({"predicted label": "functional"}, inplace=True)

In [160]:
df_submission.to_csv("/content/gd.csv", index= False)